# Exercise 5
- Required packages:
    - `Plots.jl` (for plotting)
    - `ExcelReaders.jl` (to read excel file)

In [1]:
using Plots
pyplot()
using ExcelReaders

## Functions for Hodrick-Prescott (HP) filter
This exercise allows us to showcase how different methods of computing the HP filter affects runtime and memory use.

- First, we'll use the loop method with a dense matrix to create matrix $H$.
- Second, we'll do a loop again, but with a sparse matrix.
- Third, we'll vectorize the calculation with a dense matrix and via the function `spdiagm` to create matrix $H$.
- Finally, we'll vectorize the calculation with a sparse matrix and via the function `spdiagm` to create matrix $H$.

### Declare the matrices

In [2]:
abstract type MatrixType end
struct Dense <: MatrixType end
struct Sparse <: MatrixType end
abstract type AssignMethod end
struct Loop <: AssignMethod end
struct Vectorized <: AssignMethod end

### Loop version

In [3]:
function create_matrix(λ, N::Integer, ::Dense, ::Loop)
    H = zeros(N, N)
    assign!(H, λ, N)
    return H
end;

function create_matrix(λ, N::Integer, ::Sparse, ::Loop)
    H = spzeros(N, N)
    assign!(H, λ, N)
    return H
end;

function assign!(H::AbstractMatrix, λ, N) 
    for j = 1:N
        for i = 1:N
            if j == 1
                H[1, 1] = 1+λ;
                H[2, 1] = -2λ;
                H[3, 1] = λ;
            elseif j == 2
                H[1, 2] = -2λ;
                H[2, 2] = 1 + 5λ;
                H[3, 2] = -4λ;
                H[4, 2] = λ;
            elseif j == N - 1
                H[N - 3, N - 1] = λ;
                H[N - 2, N - 1] = -4λ;
                H[N - 1, N - 1] = 1 + 5λ;
                H[N,     N - 1] = -2λ;
            elseif j == N
                H[N - 2, N] = λ;
                H[N - 1, N] = -2λ;
                H[N,     N] = 1+λ;
            else
                H[j - 2, j] = λ;
                H[j - 1, j] = -4λ;
                H[j,     j] = 1 + 6λ;
                H[j + 1, j] = -4λ;
                H[j + 2, j] = λ;
            end
        end
    end
end;

### Vectorized version

In [4]:
function create_matrix(λ::Real, N::Integer, ::Sparse, ::Vectorized) 
    return spdiagm(-2 => fill(λ, N-2),
                    -1 => vcat(-2λ, fill(-4λ, N - 3), -2λ),
                     0 => vcat(1 + λ, 1 + 5λ, fill(1 + 6λ, N-4),
                               1 + 5λ, 1 + λ),
                     1 => vcat(-2λ, fill(-4λ, N - 3), -2λ),
                     2 => fill(λ, N-2))
end;

function create_matrix(λ::Real, N::Integer, ::Dense, ::Vectorized)
    H = zeros(N, N)
    H += diagm(fill(λ, N-2), -2)
    H += diagm(vcat(-2λ, fill(-4λ, N - 3), -2λ), -1)
    H += diagm(vcat(1 + λ, 1 + 5λ, fill(1 + 6λ, N-4),
                           1 + 5λ, 1 + λ), 0)
    H += diagm(vcat(-2λ, fill(-4λ, N - 3), -2λ), 1)
    H += diagm(fill(λ, N-2), 2)
    return H
end;

In [5]:
function hp_filter(y::AbstractVector{T}, λ::Real, mt::MatrixType, am::AssignMethod) where T <: Real
    N = length(y)
    H = create_matrix(T(λ), N, mt, am)
    y_trend = H \ y
    y_cyclical = y - y_trend
    return y_trend, y_cyclical
end;

## Apply HP filter

*Note*: First run for each include compilation time, so don't take it seriously. Run a second time to get a better sense of the differences.

In [6]:
data = readxlsheet("data/US_Data.xlsx", "Data");
y = collect(data[4:end, 2]); # removing header
T = length(y);

# Looped dense version
println("Looped dense")
@time (ytr1600, yc1600) = hp_filter(Float64.(y), 1600, Dense(), Loop());
@time (ytr1e5, yc1e5)   = hp_filter(Float64.(y), 1e5, Dense(), Loop());

# Looped sparse version
println("Looped sparse")
@time (ytr1600, yc1600) = hp_filter(Float64.(y), 1600, Sparse(), Loop());
@time (ytr1e5, yc1e5)   = hp_filter(Float64.(y), 1e5, Sparse(), Loop());

# Dense matrix version
println("Vectorized dense")
@time (ytr1600, yc1600) = hp_filter(Float64.(y), 1600, Dense(), Vectorized());
@time (ytr1e5, yc1e5)   = hp_filter(Float64.(y), 1e5, Dense(), Vectorized());

# Sparse matrix version
println("Vectorized sparse")
@time (ytr1600, yc1600) = hp_filter(Float64.(y), 1600, Sparse(), Vectorized());
@time (ytr1e5, yc1e5)   = hp_filter(Float64.(y), 1e5, Sparse(), Vectorized());

Looped dense
  5.267661 seconds (1.31 M allocations: 69.188 MiB, 6.57% gc time)
  0.048105 seconds (998 allocations: 954.137 KiB)
Looped sparse
  1.623298 seconds (250.93 k allocations: 13.624 MiB, 2.09% gc time)
  0.025615 seconds (1.21 k allocations: 255.492 KiB)
Vectorized dense
  1.128149 seconds (288.31 k allocations: 20.795 MiB, 3.79% gc time)
  0.024424 seconds (1.24 k allocations: 5.349 MiB)
Vectorized sparse
  3.065138 seconds (1.49 M allocations: 81.303 MiB, 3.46% gc time)
  0.014130 seconds (1.39 k allocations: 284.764 KiB)


## With large data

In [ ]:
N_test = 5_000
y_test = randn(N_test)

# Looped dense version
println("Looped dense")
@time (ytrtest_del, yctest_del) = hp_filter(y_test, 1600, Dense(), Loop());

# Looped sparse version
println("Looped sparse")
@time (ytrtest_spl, yctest_spl) = hp_filter(y_test, 1600, Sparse(), Loop());

# Dense matrix version
println("Vectorized dense")
@time (ytrtest_dev, yctest_dev) = hp_filter(y_test, 1600, Dense(), Vectorized());

# # Sparse matrix version
println("Vectorized sparse")
@time (ytrtest_spv, yctest_spv) = hp_filter(y_test, 1600, Sparse(), Vectorized());

Looped dense
  8.708596 seconds (27 allocations: 381.585 MiB, 5.26% gc time)
Looped sparse
 

## Plot trend component
Now back to the actual assignment.

In [ ]:
plot([log.(ytr1600), log.(ytr1e5), log.(y)], lw = 2,
     lab = ["λ = 1600" "λ = 10⁵" "Raw Data"],
     title = "log of real GDP")

## Plot cyclical component

In [ ]:
p = plot([yc1600, yc1e5], lw = 2,
         lab = ["λ = 1600" "λ = 10⁵" "Raw Data"],
         title = "log of real GDP")